In [1]:
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
import math
load_dotenv()
base_path = Path.home() / Path(os.environ.get("DATA_PATH"))
print(base_path)

df_50 = pd.read_csv('slate_recommendation_50.csv')
df_200 = pd.read_csv('slate_recommendation_200.csv')

/home/aayush/rsys_data


In [2]:
print(df_50['precision'].mean())
print(df_200['precision'].mean())

0.7395823529411765
0.2843292307692308


In [3]:
print(df_50['recall'].mean())
print(df_200['recall'].mean())

0.18292647058823527
0.04790461538461538


In [4]:

positive_precision_50 = (df_50['precision'] > 0).sum()
positive_precision_200 = (df_200['precision'] > 0).sum()

total_rows_50 = len(df_50)
total_rows_200 = len(df_200)

percentage_positive_50 = (positive_precision_50 / total_rows_50) * 100
percentage_positive_200 = (positive_precision_200 / total_rows_200) * 100

print(f"slate_recommendation_50.csv: {percentage_positive_50:.2f}% rows have precision > 0")
print(f"slate_recommendation_200.csv: {percentage_positive_200:.2f}% rows have precision > 0")


slate_recommendation_50.csv: 97.06% rows have precision > 0
slate_recommendation_200.csv: 73.85% rows have precision > 0


In [5]:
df = pd.read_csv(
   base_path/ Path("behaviors.tsv"), 
    sep="\t",
    names=["impressionId","userId","timestamp","click_history","impressions"])

print(f"The dataset originally consist of {len(df)} number of interactions.")

news = pd.read_csv(
    base_path/ Path("news.tsv"), 
    sep="\t",
    names=["itemId","category","subcategory","title","abstract","url","title_entities","abstract_entities"])
print(f"The article data consist in total of {len(news)} number of articles.")
categories = news['category'].unique()
news["num_category"] = news["category"].factorize()[0]
article_category = news.set_index("itemId")["num_category"].to_dict()
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings]
    return click,non_click


df['click'], df['presented_slate'] = zip(*df['impressions'].map(process_impression))
valid_article_ids = set(news['itemId'])
print(len(news['itemId']))
def filter_click_list(click_list):
    return [item for item in click_list if item in valid_article_ids]
def filter_click_history_list(click_list):
    if pd.notna(click_list['click_history']):
     return [item for item in click_list['click_history'].split() if item in valid_article_ids]
    else:
     return []
behaviors_df=df

behaviors_df['click'] = behaviors_df['click'].apply(filter_click_list)
behaviors_df['presented_slate'] = behaviors_df['presented_slate'].apply(filter_click_list)
behaviors_df['click_history'] = behaviors_df.apply(filter_click_history_list,axis=1)
# behaviors_df[behaviors_df['click'].apply(lambda x: any(item in valid_article_ids for item in x))]
# filtered_behaviors_df = filtered_behaviors_df[filtered_behaviors_df['presented_slate'].apply(lambda x: any(item in valid_article_ids for item in x))]
filtered_behaviors_df = behaviors_df[behaviors_df['click'].apply(lambda x: len(x) > 0)]
filtered_behaviors_df

The dataset originally consist of 2232748 number of interactions.
The article data consist in total of 101527 number of articles.
101527


,impressionId,userId,timestamp,click_history,impressions,click,presented_slate
0,1,U87243,11/10/2019 11:30:54 AM,"[N8668, N39081, N65259, N79529, N73408, N43615...",N78206-0 N26368-0 N7578-0 N58592-0 N19858-0 N5...,"[N94157, N78699, N71090, N31174]","[N78206, N26368, N7578, N58592, N19858, N58258..."
1,2,U598644,11/12/2019 1:45:29 PM,"[N56056, N8726, N70353, N67998, N83823, N11110...",N47996-0 N82719-0 N117066-0 N8491-0 N123784-0 ...,"[N25587, N36266]","[N47996, N82719, N117066, N8491, N123784, N214..."
2,3,U532401,11/13/2019 11:23:03 AM,"[N128643, N87446, N122948, N9375, N82348, N129...",N103852-0 N53474-0 N127836-0 N47925-1,[N47925],"[N103852, N53474, N127836, N47925]"
3,4,U593596,11/12/2019 12:24:09 PM,"[N31043, N39592, N4104, N8223, N114581, N92747...",N38902-0 N76434-0 N71593-0 N100073-0 N108736-0...,[N114935],"[N38902, N76434, N71593, N100073, N108736, N30..."
4,5,U239687,11/14/2019 8:03:01 PM,"[N65250, N122359, N71723, N53796, N41663, N414...",N76209-0 N48841-0 N67937-0 N62235-0 N6307-0 N3...,[N86258],"[N76209, N48841, N67937, N62235, N6307, N34254..."
...,...,...,...,...,...,...,...
2232743,2232744,U316192,11/13/2019 6:50:02 PM,"[N122359, N37069, N95876, N28787, N73408, N112...",N113723-0 N123683-1 N5287-0 N76677-0 N53474-0,[N123683],"[N113723, N123683, N5287, N76677, N53474]"
2232744,2232745,U451238,11/12/2019 8:54:06 AM,"[N12575, N93816, N71643, N87236, N87236]",N18861-0 N20990-0 N43085-0 N7937-1,[N7937],"[N18861, N20990, N43085, N7937]"
2232745,2232746,U151246,11/13/2019 12:42:51 PM,"[N27587, N49668]",N39887-1 N22811-0 N110709-1 N1923-0 N24001-1 N...,"[N39887, N110709, N24001, N127572, N10285]","[N39887, N22811, N110709, N1923, N24001, N7667..."
2232746,2232747,U330725,11/12/2019 1:22:57 PM,"[N121944, N91510, N42280, N60061, N63032, N125...",N18947-0 N88808-1 N10012-0 N38902-0 N33078-0 N...,[N88808],"[N18947, N88808, N10012, N38902, N33078, N1003..."


In [12]:
import ast
import pandas as pd

# Safely convert string representations of lists to actual lists for recommended_slate
def safe_list_conversion(value):
    if isinstance(value, str):
        try:
            # Parse the string to a list
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if parsing fails
    return value if isinstance(value, list) else []

# Apply conversion to ensure consistent list types
df_200['recommended_slate'] = df_200['recommended_slate'].apply(safe_list_conversion)
filtered_behaviors_df['click'] = filtered_behaviors_df['click'].apply(lambda x: x if isinstance(x, list) else [])

# Merge DataFrames on userId
merged_df = pd.merge(df_200, filtered_behaviors_df[['userId', 'click']], on='userId', how='left')

# Compute true_click: 1 if there are common elements, else 0
def compute_true_click(row):
    if isinstance(row['click'], list) and isinstance(row['recommended_slate'], list):
        return 1 if set(row['click']) & set(row['recommended_slate']) else 0
    return 0

merged_df['true_click'] = merged_df.apply(compute_true_click, axis=1)

# Update df_200 with the new column
df_200['true_click'] = merged_df['true_click']

print(df_200['true_click'].mean())
 # Check result


0.03076923076923077


In [13]:
import ast
import pandas as pd

# Safely convert string representations of lists to actual lists for recommended_slate
def safe_list_conversion(value):
    if isinstance(value, str):
        try:
            # Parse the string to a list
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return []  # Return an empty list if parsing fails
    return value if isinstance(value, list) else []

# Apply conversion to ensure consistent list types
df_50['recommended_slate'] = df_50['recommended_slate'].apply(safe_list_conversion)
filtered_behaviors_df['click'] = filtered_behaviors_df['click'].apply(lambda x: x if isinstance(x, list) else [])

# Merge DataFrames on userId
merged_df = pd.merge(df_50, filtered_behaviors_df[['userId', 'click']], on='userId', how='left')

# Compute true_click: 1 if there are common elements, else 0
def compute_true_click(row):
    if isinstance(row['click'], list) and isinstance(row['recommended_slate'], list):
        return 1 if set(row['click']) & set(row['recommended_slate']) else 0
    return 0

merged_df['true_click'] = merged_df.apply(compute_true_click, axis=1)

# Update df_50 with the new column
df_50['true_click'] = merged_df['true_click']

print(df_50['true_click'].mean())




0.14705882352941177
